# Subprocess  WindowsError 5

场景是这样的,有一个`app`目录,里面有一个主程序`main.py`, 主程序会调用`app`目录外的`updater.py`对`app`目录进行升级:
- 下载新的包
- 重命名`app`为`app_old`
- 将新的包拷进来，成为新的`app`目录
- 删除`app_old`

说明: `main.py`和`updater.py`会使用`pyinstaller`打包成`exe`

但是在执行`os.rename`对目录进行重命名时发生错误了:
```
WindowsError: [Error 5] 
```


## 问题模拟

#### app/main.py代码

主程序代码,启动时会打开一个日志文件. 然后调用`updater.exe`程序.


In [ ]:
# encoding: utf-8
import logging
import os
import subprocess
import sys

CUR_DIR = os.path.dirname(os.path.abspath(sys.argv[0]))

logging.basicConfig(filename=os.path.join(CUR_DIR, "app.log"),
                    filemode="w",level=logging.INFO,
                    format='%(asctime)s  [%(levelname)s]- %(message)s')
UPDATER = os.path.abspath(os.path.join(CUR_DIR, "..", "updater.exe"))


def main():
    subprocess.Popen([UPDATER])
    sys.exit()


if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        pass


####  updater.py代码

启动后会对`app`目录重命名为`app_old`


In [ ]:
# -*- encoding: utf-8 -*-
import logging
import os
import sys
import zipfile

CUR_DIR = os.path.dirname(os.path.abspath(sys.argv[0]))

CACHE_DIR = os.path.join(CUR_DIR, "cache")
NEW_PACKAGE = os.path.join(CUR_DIR, "app.zip")

logging.basicConfig(filename=os.path.join(CUR_DIR, "updater.log"), filemode="w", level=logging.INFO,
                    format='%(asctime)s  [%(levelname)s]- %(message)s')


def update_files():
    logging.info(u"删除缓存目录:%s", CACHE_DIR)
    with zipfile.ZipFile(NEW_PACKAGE) as zip:
        zip.extractall(CACHE_DIR)
    
    app_cache_dir = os.path.join(CUR_DIR, "cache", "app")
    app_dir = os.path.join(CUR_DIR, "app")
    app_old_dir = os.path.join(CUR_DIR, "old_app")
    logging.info(u"替换目录%s 为 %s", app_dir, app_old_dir)
    os.rename(app_dir, app_old_dir)


if __name__ == "__main__":
    
    try:
        update_files()
    
    except Exception as e:
        logging.exception(e)


#### build.py 代码

- 将`main.py`打包成`exe`并复制到`main.py`相同的目录
- 将`updater`打包成`exe`并复制到`updater.py`相同目录

注意: 需要安装`pyinstaller`


In [ ]:


import os
import shutil
import subprocess

CUR_DIR = os.path.abspath(os.path.dirname(__file__))

APP_DIR = os.path.join(CUR_DIR, "app")

pyinstaller_exe = r"C:\Python27\Scripts\pyinstaller.exe"

try:
    shutil.rmtree(os.path.join(CUR_DIR, "build"))
    shutil.rmtree(os.path.join(CUR_DIR, "dist"))
except:
    pass
subprocess.check_call("{pyinstaller} -w -F --onefile  --clean {script}".format(
    pyinstaller=pyinstaller_exe,
    script=os.path.join(CUR_DIR, "updater.py")
), shell=True, cwd=CUR_DIR)

shutil.copy(os.path.join(CUR_DIR, "dist", "updater.exe"),
                CUR_DIR)

try:
    shutil.rmtree(os.path.join(APP_DIR, "build"))
    shutil.rmtree(os.path.join(APP_DIR, "dist"))
except:
    pass
subprocess.check_call("{pyinstaller} -w -F --onefile  --clean {script}".format(
    pyinstaller=pyinstaller_exe,
    script=os.path.join(APP_DIR, "main.py")
), shell=True, cwd=APP_DIR)

shutil.copy(os.path.join(APP_DIR, "dist", "main.exe"),
                APP_DIR)


## 结果

```
2017-03-24 17:47:13,496  [ERROR]- [Error 5] 
Traceback (most recent call last):
  File "books\bug\subprocess_windowserror_5\updater.py", line 31, in <module>
  File "books\bug\subprocess_windowserror_5\updater.py", line 25, in update_files
WindowsError: [Error 5] 
```

## 问题分析